In [1]:
from enum import Enum
from collections import Counter
from tqdm import tqdm

In [33]:
aa = {"a": 1, "b": 2}
b = aa.pop("a")

In [35]:
aa

{'b': 2}

In [72]:
class HandType(Enum):
    HIGH_CARD = 0
    ONE_PAIR = 1
    TWO_PAIR = 2
    THREE_OF_A_KIND = 3
    FULL_HOUSE = 4
    FOUR_OF_A_KIND = 5
    FIVE_OF_A_KIND = 6

class Hand:
    def __init__(self, cards: str, part: int = 1):
        self.cards = cards
        self.part = part
        if self.part == 1:
            self.ordered_cards = "A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, 2".split(', ')
        else:
            self.ordered_cards = "A, K, Q, T, 9, 8, 7, 6, 5, 4, 3, 2, J".split(', ')
        
    def __repr__(self):
        return f"Hand('{self.cards}')"
    
    @classmethod
    def from_str(cls, value: str):
        return cls(cards=value)
    
    @property
    def cards_counter(self):
        base_counter = Counter(self.cards)
        if self.part == 2 and "J" in self.cards:
            # replace J key by the highest card
            count_j = base_counter.pop("J")
            if len(base_counter) > 0:
                max_count = max(base_counter.values())
                max_cards = [u for u, v in base_counter.items() if v==max_count]
                ref_card = sorted(max_cards, key=lambda x: self.ordered_cards.index(x))[0]
                base_counter[ref_card] += count_j
            else:
                base_counter["A"] = count_j
        return base_counter
    
    @property
    def card_strenghts(self) -> list[int]:
        return [(len(self.ordered_cards) - self.ordered_cards.index(i)) for i in self.cards]
    
    @property
    def hand_type(self) -> HandType:
        cards_counter = self.cards_counter
        max_cards = max(cards_counter.values())
        if max_cards == 1:
            return HandType.HIGH_CARD
        elif max_cards == 2:
            # check if there's 1 or two pairs
            n_pairs = [t for t in cards_counter.values() if t == 2]
            if len(n_pairs) == 2:
                return HandType.TWO_PAIR
            else:
                return HandType.ONE_PAIR
        elif max_cards == 3:
            if 2 in cards_counter.values():
                # we got a pair too
                return HandType.FULL_HOUSE
            else:
                return HandType.THREE_OF_A_KIND
        elif max_cards == 4:
            return HandType.FOUR_OF_A_KIND
        elif max_cards == 5:
            return HandType.FIVE_OF_A_KIND
        
    def __lt__(self, other: 'Hand'):
        if self.hand_type.value < other.hand_type.value:
            return True
        elif self.hand_type.value == other.hand_type.value:
            for i in range(len(self.cards)):
                if self.card_strenghts[i] < other.card_strenghts[i]:
                    return True
                elif self.card_strenghts[i] == other.card_strenghts[i]:
                    continue
                else:
                    return False
        else:
            return False
        
    def __eq__(self, other: 'Hand'):
        return self.cards == other.cards

        
class HandBid:
    def __init__(self, hand: Hand, bid: int):
        self.hand = hand
        self.bid = bid
        
    def __repr__(self) -> str:
        return f"HandBid(hand='{self.hand}', bid=({self.bid}))"
    
    def __lt__(self, other: 'HandBid') -> bool:
        return self.hand < other.hand
    
    def __eq__(self, other: 'HandBid') -> bool:
        return self.hand == other.hand

    @classmethod
    def from_str(cls, value: str, part: int = 1) -> 'HandBid':
        hand, bid = value.split(' ')
        return cls(hand=Hand(cards=hand, part=part), bid=int(bid))
        
class CamelGame:
    def __init__(self, hands: list[HandBid]):
        self.hands = hands
    
    @classmethod
    def from_str(cls, value: str, part: int = 1) -> 'CamelGame':
        return cls(
            hands = [HandBid.from_str(t, part=part) for t in value.strip().split('\n')]
        )
        
    @property
    def hand_ranks(self):
        return sorted(range(len(self.hands)), key=self.hands.__getitem__)
    
    def print_hands(self):
        hand_ranks = self.hand_ranks
        for i in tqdm(range(len(self.hands))):
            rank = hand_ranks[i] + 1
            hand = self.hands[i].hand
            print(f"{hand} | {hand.hand_type} ({hand.hand_type.value}) | {rank}")
    
    def solution_part_1(self):
        out = 0
        sorted_hands = sorted(self.hands)
        for i, hand in tqdm(enumerate(sorted_hands), total=len(sorted_hands)):
            rank = i + 1
            bid = hand.bid
            out += rank * bid
            
        return out

In [73]:
puzzel_str = """
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
"""

In [74]:
game = CamelGame.from_str(puzzel_str, part=2)
game.solution_part_1()

100%|██████████| 5/5 [00:00<00:00, 33770.56it/s]


5905

In [75]:
puzzel_str = """
TQ5TT 421
65K2J 973
K543T 50
AT68Q 952
JKTTJ 766
55Q55 860
J9427 977
K36QA 569
TQQJ2 710
KK274 900
78667 841
3J3A3 753
35333 832
4T4TT 526
5JJ38 283
3J999 48
94549 220
TJ777 917
8483J 509
J6628 103
Q7999 423
6AQTT 711
Q77Q7 914
4K44T 898
J87A7 793
KKK55 750
KK336 142
T6JJT 410
58Q44 313
Q8959 41
89989 224
27598 271
KKJ77 948
27544 751
239TA 384
7396K 937
J7JQT 493
29Q3K 735
83645 367
755AA 112
J43K5 465
Q6T85 830
Q7J35 360
28877 58
6TJ54 173
7Q4KQ 969
8AA99 70
256TT 216
K4759 722
T9559 259
8888J 53
J774Q 542
726K8 528
88KKK 156
6JTQ3 326
KA26K 515
23299 311
44445 179
92995 890
2KTT2 337
JAAJA 452
7J777 922
945KQ 49
J32A8 996
7JK62 376
96555 248
AAA9A 201
4K5T5 643
97665 993
7KJ22 510
KAKAK 913
444KK 101
AK2J4 736
Q265T 902
K4K2K 846
TTAKT 769
8T53J 433
4Q6TT 661
TTTJ5 381
JJ52T 822
93898 538
74944 966
3K33K 1
K3J33 61
68JJT 634
JT8TQ 15
7T777 327
483A2 836
6666J 445
A7A44 968
55252 992
QQQ36 94
3AA3A 457
965J6 485
2828J 960
J7K79 434
54J5T 979
6ATKJ 552
TTT4T 627
6566J 598
K465K 397
K3823 568
TA3A7 573
58558 729
22A22 699
54548 854
74869 82
337QQ 482
5QTK8 306
A2A77 355
678QA 998
K9KKK 855
2T5J9 68
6QQK6 647
5QJJQ 924
57779 414
55565 885
T3Q4A 834
AAQ8A 366
7TK7K 873
92Q68 508
94K86 856
A42A7 145
7J6T4 113
52QQQ 148
2QA58 791
QT427 284
22929 35
75777 480
4TKKK 54
66KK6 333
26K9T 351
J7T7T 869
K222J 349
5JQT6 504
5679J 143
4JT74 464
AJ885 681
9J575 345
97727 191
2K277 730
22229 507
38333 74
667TJ 877
QKJ4T 10
5Q4QJ 618
KKK77 483
65643 619
86QTJ 415
ATAA5 175
3T343 435
9Q9QQ 700
44332 884
98847 843
455A5 500
6438T 622
88T84 289
K4444 579
59KK5 136
K999K 357
4T6TT 116
9636T 531
44AJ8 453
T7JJ4 991
JT88T 393
257KA 932
Q7727 85
39A95 675
6K4KK 670
9J549 492
QJ6A2 527
99989 728
J992T 570
JJJKK 606
AJ933 323
QAQ68 228
KAKKJ 800
A55QA 356
3366J 896
T2KK2 705
76J6A 408
3K7J6 299
96QJA 215
7A8Q9 89
TQQJ5 310
TQ76A 472
99T9T 129
6JQ2T 870
7977T 450
9AT46 505
2T9JQ 651
3AJAA 544
QQ6AJ 625
36823 170
56567 429
99599 758
AA2A2 620
222J7 611
88858 8
363A3 139
AJ977 665
JQQQ2 83
8727A 815
KKJ53 335
6Q6QQ 676
56T5T 425
KK7T3 792
79655 252
37723 40
KJKK5 881
8QQQ6 458
JKQJK 240
2J6TJ 257
3JQT3 925
39A99 399
3J95K 470
TJ565 861
8TAKK 473
33633 739
5AT9Q 172
3628Q 25
KKKA6 463
Q2984 32
JK93Q 978
8JJJJ 724
49997 852
TT22T 484
J75T5 137
25656 16
33933 654
8863T 576
4A99A 771
4J789 687
9992Q 127
248T6 561
55J5J 857
6666A 980
JAQQ9 212
2KJ34 151
A9A6A 340
5KKK3 889
J2J22 99
69999 4
ATAAJ 451
4KAA4 109
88838 797
QJQJT 849
88849 448
J9K28 596
A6996 177
Q3T44 646
AQ28K 182
7A339 549
JK6K6 853
T22J3 671
55KAA 14
2J5T2 637
JKK9J 835
33387 39
2QK2K 419
T75A8 352
32332 910
68Q94 577
89873 467
8J788 633
T8K57 645
255A4 342
73443 802
655J6 814
24442 34
5TTQ5 763
32349 747
42249 795
TJ4Q3 499
KKKK3 686
K5555 765
QQQQ4 86
QJ669 407
86A42 65
93239 155
44777 777
2Q242 23
45A79 217
4J7KT 744
5QA55 244
7775J 200
JJQQQ 801
47777 263
7782K 779
92Q9K 787
J93A2 506
4JQQQ 105
79K29 459
54J9J 124
6T8Q2 338
33J3J 286
JK566 78
9255A 455
JT2KJ 120
QQKKQ 781
9QQ99 589
A4KK4 100
2A8Q6 270
8J99A 117
338T8 430
55544 369
2KK2K 726
994K4 447
42J24 282
79643 107
9K776 920
A27A9 164
6Q266 461
5TTT5 308
K8888 51
7AKT6 181
Q4852 893
K6KKK 888
J6966 353
J7672 307
A8JQ8 714
4QK44 43
24822 592
JA2QA 521
Q5853 75
7AA2J 535
258K5 420
T4824 545
5QT88 985
Q959J 171
AT874 556
84J39 380
3Q27A 691
AT624 944
88868 511
TAAAT 226
AJA2J 55
676J5 685
97999 693
27277 895
48699 648
AAKK8 3
KKKJK 486
6A663 131
465J8 386
86886 754
AKA8A 819
3QQ3T 891
47T47 62
AAK2A 426
49J22 983
QJ8AQ 245
39AK3 385
8493K 866
AA933 174
5J525 560
A2552 22
4T4A9 383
Q76K6 525
66686 478
37J3Q 303
Q52QT 147
QQ984 168
667J6 293
3J333 158
KKQTK 469
T2JT6 839
9T675 401
8J7Q4 38
4A5Q6 247
6KJ34 180
55585 404
TA484 512
A4A4A 36
642QQ 199
Q48AJ 593
Q5347 479
Q9Q63 395
84278 848
95KQT 184
J444J 903
QA97J 770
66266 256
94J44 967
QJ674 971
T498T 72
A66J5 823
43434 91
999J9 52
8Q88J 361
TTT8T 391
33332 206
424J8 442
78222 784
4253A 712
KT677 683
QK63Q 938
42224 344
27268 698
TJ323 471
2Q2Q5 582
85595 406
222TK 662
89888 166
63J3A 867
T28T5 439
JQ4TQ 13
44J44 123
99J77 211
5JAQA 524
6AKAA 696
K77J7 87
TJKQT 287
5TJ77 578
JT55A 276
J668J 474
234T5 953
3T3K3 30
K4AAJ 312
T2TK4 27
KAT35 929
54A26 692
33888 514
36233 842
J27Q6 631
A5J55 329
AT692 616
8QQ8Q 368
K9K5K 118
6886T 315
22227 185
54Q55 346
36KA7 972
Q77JQ 831
J399K 424
55455 239
56446 904
3TJ7J 47
2K224 961
A5T8K 775
K44Q5 443
54544 121
8AA4J 964
89Q68 440
K7KKK 721
QK82T 176
Q47A2 868
TT756 595
K7QK7 436
9QQQQ 304
3JAJK 657
3AJAQ 392
K5KK6 603
Q85Q4 57
JKTQJ 456
7A777 709
TT885 614
7737J 298
T6TA6 963
92T43 249
57A9Q 879
K8K83 97
22AA2 790
2K584 785
446J6 138
KTTTT 441
Q85AQ 669
777K7 12
4A442 339
K7Q9A 638
9AA9T 759
7T6TT 93
44JT4 776
6T996 805
2225K 516
5Q9K2 804
26286 612
J9K57 222
9QQ2Q 639
96T3J 768
J4244 162
22352 641
42262 820
AAAA5 982
A23JQ 402
A2259 833
722AA 912
KKJK2 752
74773 951
5A5A5 649
J94TK 413
K86J3 812
3T776 37
AA2K2 438
77772 227
T8T7J 732
5QQ85 269
QQJ9Q 291
72TJT 761
3A8AA 275
K73Q2 734
433Q3 387
22454 491
J844T 597
754AT 584
7T5K2 731
AKKA5 911
TT9TT 617
T4QQT 161
66677 742
QQKQQ 587
9TJ8K 277
AAJT8 71
6J9KQ 213
84843 230
A7A2A 567
89692 267
AT78J 828
49998 864
677Q6 396
4323J 536
7JJ47 328
J78T8 827
97977 591
7727A 562
JTTTT 225
J888K 466
JQ38K 495
K77T7 679
8QJ6J 119
82A88 444
K5K55 984
99KKK 140
66Q6Q 572
62KK2 67
T4K28 809
QKK4K 66
55KT2 104
78ATT 837
58ATT 845
Q4QQ8 623
2KA25 636
2TT22 723
QQQ6Q 379
46A5J 554
49A2Q 241
22T22 219
37336 343
67474 642
T8TTJ 673
399Q3 258
QJQ88 382
3683T 503
A2QT8 906
34333 886
44442 936
K292K 816
JK622 601
Q7Q6J 321
AA492 31
A8A88 630
TKJ67 26
642QJ 537
29993 624
22666 581
TT99J 957
4Q9TT 9
276A9 29
J6282 152
A8672 557
8AAAA 551
TTT23 548
65Q73 946
4JKKJ 64
68286 208
93688 737
4A444 517
AA9A3 540
A4858 943
J3J92 427
AA77A 905
KKJ99 743
6TT66 954
99499 844
66692 640
8TTJ3 682
6J558 659
A9AAQ 462
K9294 481
888J6 354
88288 388
67767 923
55425 292
T227Q 60
4744T 798
7Q777 564
43343 865
3T3TT 916
K83QQ 811
8J885 908
Q747Q 772
46J63 871
AAAAJ 817
2622J 183
7T924 246
Q3K26 56
32222 716
A95T2 125
4965Q 431
3Q89Q 251
9754Q 59
4TTTK 919
QQQ5T 24
99444 268
33356 608
AAA4A 794
56569 238
42T42 918
TJQJ3 69
A4449 159
45K55 42
T5553 976
K9899 297
64JJA 859
456JK 958
8QQQJ 688
J8824 989
27J9T 416
55J32 941
6666K 695
AJ8AA 532
38846 371
46T8J 628
QAAQ8 196
7TTT7 588
48487 88
88848 331
K3333 300
5489J 363
Q7358 207
J4JTK 931
2K9KK 316
92J95 44
35553 280
2829J 210
ATJT6 7
AA9KQ 613
86766 390
8QQ88 558
29992 858
J6QQQ 585
8KT8T 672
52355 708
Q7TJ5 320
89996 167
838JT 702
K565K 757
KQ3KJ 334
8JJ99 965
QQQQ8 994
3653K 496
J8867 460
KQ729 336
777AA 110
777T4 935
84K6J 309
2JT7A 398
T5J67 272
8A222 372
6TT77 45
93QTQ 945
Q4399 95
KKA4K 988
8AATK 718
QKQJK 974
23284 764
J9QJT 876
7J869 281
KK466 847
J3Q9T 678
QA7A9 740
A298K 141
J5ATA 840
5J44J 488
76386 160
2222J 279
AA66A 813
A63K4 892
JA999 264
9K39K 189
J3737 132
65332 796
K3383 295
654K3 660
49552 713
QT7JQ 232
9JK92 377
9JK9A 253
5QQ98 897
JK23T 652
39J77 810
A636A 574
555J5 586
K2836 157
J9839 243
AAQQJ 604
33J83 719
6AJAA 301
94483 755
65665 997
8TJ88 250
9JAAJ 108
TTT5T 899
9A9AA 600
43J3T 144
JT422 653
TJJTT 432
AAAAT 19
KK5KK 135
JKKTJ 863
5JA86 725
4Q5KQ 664
QKQJQ 663
66K33 128
93365 981
A2QQ8 273
474K5 347
84TTT 733
7K333 389
Q24K5 782
77965 534
8Q43J 894
JA888 523
KK4K4 122
TT88T 205
77J55 599
22JJ9 566
39989 756
79J9T 555
A792Q 111
5QQ5Q 825
98985 322
878KK 218
9A5A3 609
K5QK5 378
5483K 126
4J744 519
4J295 501
JQAQQ 468
3T333 762
272QK 265
8KQ8Q 84
4KJK4 305
23QQQ 294
A4JA6 655
A44J9 332
44QQQ 449
55AAA 926
39T88 644
QQ767 807
KJJQ4 818
8T9K4 130
23277 927
758JA 73
77KK7 543
88222 529
T783T 319
AAQAA 403
K9J22 550
AJ338 28
23888 1000
5JA34 872
22664 261
55765 520
79927 947
2J367 487
8J8J8 236
7A7QQ 704
K5998 489
824K9 774
Q2974 231
37J4A 607
2J727 887
65556 738
6KKJA 476
QKK88 165
28885 6
Q6655 115
67A66 940
822J2 942
T3T38 477
KKKTK 133
99939 907
T9TT6 302
99669 783
6J66J 223
T73J9 400
5TA99 79
2T339 193
TA87Q 745
24798 565
9AJ5Q 933
TAA2T 437
2J229 956
4AKJK 498
66449 412
8J4TT 826
7JK68 934
7JJ2Q 494
QT4AA 786
555T6 987
8848K 348
K884K 255
94256 717
669Q6 513
49844 986
A7T7A 11
QTTTT 629
6TT6T 237
4948J 883
KKK8K 658
5525T 950
83475 959
JAT4K 778
JJ777 314
KKKJJ 594
66656 677
3Q298 20
AAAA7 446
8QQ67 285
Q22QQ 939
26222 635
78Q77 146
73T33 590
QKKQK 497
4Q464 198
8888Q 547
59955 186
7A569 727
2T5K9 803
K33J7 76
99T44 365
A88AJ 454
36646 373
6499Q 428
QTTT4 98
7829Q 680
A4K2T 409
8JAK4 202
AK8A6 17
77224 278
TQQQQ 190
6KJ35 358
A535A 874
47KKK 666
ATTJA 221
QQ8Q3 571
K6AKT 163
AKAJA 502
4AJAA 760
AAKAA 715
5AJA7 324
9966Q 154
878JK 748
9499J 626
J48A8 233
4J2T9 330
JQQQQ 921
74292 741
JJJJJ 90
82T8K 559
665K7 46
99AA9 995
KK3JK 999
3K857 563
88788 417
38KT2 262
QA9QQ 720
Q72Q2 575
8KK2K 990
TT6TT 808
68959 882
44Q74 2
K3T98 824
76732 955
JQJQJ 350
93J39 780
J2J42 862
2JQT5 296
A84AA 80
5445K 235
TATTA 656
87J44 153
JAA4J 788
4QKTQ 422
7TJ34 187
2AAAA 209
93343 188
87788 749
73K83 901
53353 194
6Q6QA 288
23KTK 829
AKTKT 374
2QT94 553
2QK6K 533
35AQ3 341
J6Q84 707
5Q683 799
J3422 411
7K393 668
6JKQQ 318
9599J 5
44282 694
9J88A 150
66Q76 975
KK2KK 915
42222 18
5J3J6 851
6J767 650
66635 674
96777 962
35K3K 632
3353J 106
87777 684
5KQ55 909
Q999A 850
7KKK2 290
A4822 317
J55TT 546
Q8559 767
QTQ44 149
Q22Q2 21
K7K6T 475
57KAJ 949
22552 706
TKJ95 838
99T94 359
76266 114
99797 490
92999 697
Q3QQQ 234
5A99A 405
58T87 364
J76QK 689
574K5 274
KKAKK 370
3K3KK 81
Q2828 539
49J74 203
73425 102
4J433 169
92K4T 583
26QA5 195
Q42TT 773
9A8T2 930
A7AQ3 701
QQQQ5 92
K66KK 214
22K22 746
A7757 33
J4Q46 77
66Q66 362
898J8 541
22772 875
99333 394
AA9A2 266
46544 375
7774J 242
55QJ5 880
J4966 703
AKK4A 530
AA9TT 260
6KQJT 518
23322 610
85J33 602
K472J 192
A698K 690
4J462 134
78648 522
38533 605
22766 418
QQQQ2 197
33QQ3 204
6QQQ5 63
59KKA 970
42A22 229
29628 580
5AA3A 928
KQQQ2 615
76229 325
T3636 254
86KA2 821
88K57 806
K8652 878
A8A5A 789
ATQA3 96
25JQ7 667
74775 621
6A4A6 178
"""

In [76]:
game = CamelGame.from_str(puzzel_str, part=2)

In [79]:
game.solution_part_1()

100%|██████████| 1000/1000 [00:00<00:00, 283533.02it/s]


250057090